In [1]:
from pathlib import Path
BASE_DIR = Path("/Users/dsaluru/Desktop/GPU/bird-sql-forked/save_data/qwen2.5/")

GENERATION_DIR = BASE_DIR / "generations"
RESULTS_DIR = BASE_DIR / "with_results"
REWARD_DIR = BASE_DIR / "with_rewards"

### Generations

In [2]:
# load the generations jsonl file - data_gen.jsonl
import json

final_generations_data = {}
with open(GENERATION_DIR / "data_gen.jsonl", "r") as f:
    for line in f:
        data = json.loads(line)

        if data['question_id'] not in final_generations_data:
            final_generations_data[data['question_id']] = {
                'question_id' : data['question_id'],
                'db_id' : data['db_id'],
                'question' : data['question'],
                'evidence' : data['evidence'],
                'SQL' : data['SQL'],
                'difficulty' : data['difficulty'],
                'responses' : []
            }
        curr_responses = []
        for item in data['responses']:
            curr_responses.append(data['responses'][item]['content'])
        final_generations_data[data['question_id']]['responses'] += curr_responses




In [3]:
mean_responses = 0
for key in final_generations_data:
    mean_responses += len(final_generations_data[key]['responses'])
mean_responses /= len(final_generations_data)
print(mean_responses)


154.206


In [4]:
import pandas as pd
pd.Series(final_generations_data[1473]['responses']).value_counts()

SELECT AVG(T1.Consumption) / 12 FROM yearmonth AS T1 INNER JOIN customers AS T2 ON T1.CustomerID = T2.CustomerID WHERE T2.Segment = 'SME' AND T1.Date BETWEEN 201301 AND 201312            15
SELECT AVG(T1.Consumption) / 12 FROM yearmonth AS T1 INNER JOIN customers AS T2 ON T1.CustomerID = T2.CustomerID WHERE T1.Date BETWEEN 201301 AND 201312 AND T2.Segment = 'SME'            14
SELECT AVG(T2.Consumption) / 12 FROM customers AS T1 INNER JOIN yearmonth AS T2 ON T1.CustomerID = T2.CustomerID WHERE T1.Segment = 'SME' AND T2.Date BETWEEN 201301 AND 201312            13
SELECT AVG(T1.Consumption) / 12 FROM yearmonth AS T1 INNER JOIN customers AS T2 ON T1.CustomerID = T2.CustomerID WHERE T2.Segment = 'SME' AND T1.Date BETWEEN '201301' AND '201312'         9
SELECT AVG(T1.Consumption) / 12 FROM yearmonth AS T1 INNER JOIN customers AS T2 ON T1.CustomerID = T2.CustomerID WHERE SUBSTR(T1.Date, 1, 4) = '2013' AND T2.Segment = 'SME'                7
                                                  

### With Results

In [5]:
final_results_data = {}
with open(RESULTS_DIR / "data_with_results.jsonl", "r") as f:
    for line in f:
        data = json.loads(line)

        if data['question_id'] not in final_results_data:
            final_results_data[data['question_id']] = {
                'question_id' : data['question_id'],
                'db_id' : data['db_id'],
                'question' : data['question'],
                'evidence' : data['evidence'],
                'SQL' : data['SQL'],
                'difficulty' : data['difficulty'],
                'responses' : []
            }
        curr_responses = []
        for item in data['responses']:
            curr_responses.append((data['responses'][item]['content'], data['responses'][item]['evaluation']))
        final_results_data[data['question_id']]['responses'] += curr_responses

In [6]:
mean_responses = 0
for key in final_results_data:
    mean_responses += len(final_results_data[key]['responses'])
mean_responses /= len(final_results_data)
print(mean_responses)


83.298


In [7]:
import pandas as pd
mean_pass = 0
for key in final_results_data:
    try:
        mean_pass += pd.Series([x[1].split(':')[0] for x in final_results_data[key]['responses']]).value_counts()['pass']
    except:
        print("key : ", key)
mean_pass /= len(final_results_data)
print(mean_pass)

key :  532
key :  701
key :  704
key :  518
70.08


### with rewards

In [8]:
final_rewards_data = {}
with open(REWARD_DIR / "data_reward_all.jsonl", "r") as f:
    for line in f:
        data = json.loads(line)

        if data['question_id'] not in final_rewards_data:
            final_rewards_data[data['question_id']] = {
                'question_id' : data['question_id'],
                'db_id' : data['db_id'],
                'question' : data['question'],
                'evidence' : data['evidence'],
                'SQL' : data['SQL'],
                'difficulty' : data['difficulty'],
                'responses' : []
            }
        curr_responses = []
        for item in data['responses']:
            if data['responses'][item]['evaluation'].split(':')[0] == 'pass' and data['responses'][item]['reward'] != None:
                curr_responses.append((data['responses'][item]['content'], data['responses'][item]['evaluation'], data['responses'][item]['reward']))
        final_rewards_data[data['question_id']]['responses'] += curr_responses

In [9]:
mean_responses = 0
for key in final_rewards_data:
    mean_responses += len(final_rewards_data[key]['responses'])
mean_responses /= len(final_rewards_data)
print(mean_responses)


63.998


In [10]:
# sort the responses by reward

for key in final_rewards_data:
    final_rewards_data[key]['responses'].sort(key=lambda x: x[2], reverse=True)


In [11]:

for k in range(1, 20):
    accuracy = 0
    for key in final_rewards_data:
        all_candidates_rewards_score = [x[2] for x in final_rewards_data[key]['responses']]
        all_candidates_rewards = [x[1] for x in final_rewards_data[key]['responses']]
        all_candidates_sqls = [x[0] for x in final_rewards_data[key]['responses']]
        top_k_candidates_rewards_score = all_candidates_rewards_score[:k]
        top_k_candidates_rewards = all_candidates_rewards[:k]
        top_k_candidates_sqls = all_candidates_sqls[:k]

        if len(final_rewards_data[key]['responses']) > 0:
            if 'pass: correct' in top_k_candidates_rewards:
                accuracy += 1
    accuracy /= len(final_rewards_data)
    print("accuracy for k = ", k, " is ", accuracy)


accuracy for k =  1  is  0.69
accuracy for k =  2  is  0.712
accuracy for k =  3  is  0.716
accuracy for k =  4  is  0.722
accuracy for k =  5  is  0.732
accuracy for k =  6  is  0.744
accuracy for k =  7  is  0.748
accuracy for k =  8  is  0.752
accuracy for k =  9  is  0.758
accuracy for k =  10  is  0.758
accuracy for k =  11  is  0.762
accuracy for k =  12  is  0.772
accuracy for k =  13  is  0.78
accuracy for k =  14  is  0.78
accuracy for k =  15  is  0.78
accuracy for k =  16  is  0.788
accuracy for k =  17  is  0.79
accuracy for k =  18  is  0.792
accuracy for k =  19  is  0.796


### SQL Parsing

In [21]:
import sqlglot
from sqlglot import expressions as exp
import re

def extract_schema_linking(sql_query, dialect="sqlite"):
    try:
        # Clean up any ANSI escape codes
        clean_query = re.sub(r'\x1b\[[0-9;]*m', '', sql_query)

        parsed = sqlglot.parse_one(clean_query, dialect="sqlite")

        # Build alias mapping
        alias_to_table = {}
        for table in parsed.find_all(exp.Table):
            if table.alias:
                alias_to_table[table.alias] = table.name
            alias_to_table[table.name] = table.name

        schema_linking = {}

        # Find all column references
        for column in parsed.find_all(exp.Column):
            if column.table:
                table_name = alias_to_table.get(column.table, column.table)
                column_name = column.name

                if table_name not in schema_linking:
                    schema_linking[table_name] = set()
                schema_linking[table_name].add(column_name)

        return {table: list(columns) for table, columns in schema_linking.items()}

    except Exception as e:
        return {"error": str(e)}

In [52]:
K=50
golden_sql_parsed = {}
candidate_sql_parsed = {}
for key in final_rewards_data:
    golden_sql_parsed[key] = extract_schema_linking(final_rewards_data[key]['SQL'])

    # aggregate the sql parsed for all the responses and union them
    merged_candidate_sql_parsed = {}
    for response in final_rewards_data[key]['responses'][:K]:
        curr_sql_parsed = extract_schema_linking(response[0])
        if 'error' not in curr_sql_parsed:
            for table, columns in curr_sql_parsed.items():
                if table not in merged_candidate_sql_parsed:
                    merged_candidate_sql_parsed[table] = set()
                merged_candidate_sql_parsed[table].update(columns)

    # Convert sets back to lists
    candidate_sql_parsed[key] = {table: list(columns) for table, columns in merged_candidate_sql_parsed.items()}

# tables matches with golden sql
tables_matches = {}
tables_accuracy = 0
tables_and_fields_accuracy = 0
for key in candidate_sql_parsed:
    golden_tables = sorted(golden_sql_parsed[key].keys())
    candidate_tables = sorted(candidate_sql_parsed[key].keys())
    table_flag = True
    for table in golden_tables:
        if table not in candidate_tables:
            table_flag = False

    if table_flag:
        tables_accuracy += 1

    table_field_flag = True
    for table in golden_tables:
        if table in candidate_tables:
            golden_fields = sorted(golden_sql_parsed[key][table])
            candidate_fields = sorted(candidate_sql_parsed[key][table])
            for field in golden_fields:
                if field not in candidate_fields:
                    table_field_flag = False
                    break
        else:
            table_field_flag = False
            break
    if table_field_flag:
        tables_and_fields_accuracy += 1

In [53]:
tables_accuracy *100.0 /500

92.6

In [54]:
tables_and_fields_accuracy * 100.0 / 500

85.4

In [18]:
extract_schema_linking("SELECT COUNT(*) FROM satscores AS T1 INNER JOIN schools AS T2 ON T1.cds = T2.CDSCode WHERE T1.AvgScrMath > 400 AND T2.Virtual = 'F'")

{'satscores': ['cds', 'AvgScrMath'], 'schools': ['Virtual', 'CDSCode']}

In [14]:
[x[0] for x in final_rewards_data[5]['responses']]

["SELECT COUNT(*) FROM satscores AS T1 INNER JOIN schools AS T2 ON T1.cds = T2.CDSCode WHERE T1.AvgScrMath > 400 AND T2.Virtual = 'F'",
 "SELECT COUNT(*) FROM satscores INNER JOIN schools ON satscores.cds = schools.CDSCode WHERE schools.Virtual = 'F' AND satscores.AvgScrMath > 400",
 "SELECT COUNT(*) FROM satscores AS T1 INNER JOIN schools AS T2 ON T1.cds = T2.CDSCode WHERE T2.Virtual = 'F' AND T1.AvgScrMath > 400",
 "SELECT COUNT(T1.cds) FROM satscores AS T1 INNER JOIN schools AS T2 ON T1.cds = T2.CDSCode WHERE T1.AvgScrMath > 400 AND T2.Virtual = 'F'",
 "SELECT COUNT(*) FROM schools AS T1 INNER JOIN satscores AS T2 ON T1.CDSCode = T2.cds WHERE T1.Virtual = 'F' AND T2.AvgScrMath > 400",
 "SELECT COUNT(*) FROM schools AS T1 INNER JOIN satscores AS T2 ON T2.cds = T1.CDSCode WHERE T2.AvgScrMath > 400 AND T1.Virtual = 'F'",
 "SELECT COUNT(*) FROM schools AS T1 INNER JOIN satscores AS T2 ON T1.CDSCode = T2.cds WHERE T2.AvgScrMath > 400 AND T1.Virtual = 'F'",
 "SELECT COUNT(T1.cds) FROM sat